## Исследование данных об инвестициях венчурных фондов в компании-стартапы

В рамках данного исследования необходимо проанализировать данные о фондах и инвестициях и написать запросы к базе. База данных хранит информацию о венчурных фондах и инвестициях в компании-стартапы. Эта база данных основана на датасете Startup Investments, опубликованном на популярной платформе для соревнований по исследованию данных Kaggle. 

### Задание 1

Отобразите все записи из таблицы company по компаниям, которые закрылись.

In [ ]:
'''
SELECT COUNT(status)
FROM company
WHERE status = 'closed';
'''

### Задание 2

Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.

In [ ]:
'''
SELECT funding_total
FROM company
WHERE category_code LIKE 'news' AND country_code LIKE 'USA'
ORDER BY funding_total DESC;
'''

### Задание 3

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
'''
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code LIKE 'cash'
AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2011 AND 2013;
'''

### Задание 4

Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
'''
SELECT first_name, last_name, twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';
'''

### Задание 5

Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
'''
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';
'''

### Задание 6

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
'''
SELECT country_code, SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;
'''

### Задание 7

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
'''
SELECT funded_at, MIN(raised_amount), MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0 AND MIN(raised_amount) != MAX(raised_amount);
'''

### Задание 8

Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
'''
SELECT *,
        CASE
           WHEN invested_companies >= 100 THEN 'high_activity' 
           WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
        END
FROM fund;
'''

### Задание 9

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
'''
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));
'''

### Задание 10

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
'''
SELECT country_code, 
MIN(invested_companies),
MAX(invested_companies),
AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;
'''

### Задание 11

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
'''
SELECT first_name, last_name, instituition
FROM people
LEFT JOIN education ON people.id = education.person_id;
'''

### Задание 12

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
'''
SELECT c.name,
COUNT(DISTINCT(instituition))
FROM company AS c
LEFT JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
GROUP BY c.id
ORDER BY COUNT(DISTINCT(instituition)) DESC
LIMIT 5;
'''

### Задание 13

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
'''
SELECT DISTINCT(c.name)
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1;
'''

### Задание 14

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
'''
SELECT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT(c.id)
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1)
'''

### Задание 15

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
'''
WITH
p_id AS
(SELECT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT(c.id)
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1))

SELECT DISTINCT(person_id), instituition
FROM education AS e
JOIN p_id ON e.person_id=p_id.id
'''

### Задание 16

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
'''
WITH
p_id AS
(SELECT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT(c.id)
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1))

SELECT person_id, COUNT(instituition)
FROM education AS e
JOIN p_id ON e.person_id=p_id.id
GROUP BY person_id
'''

### Задание 17

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
'''
WITH
p_id AS
(SELECT p.id
FROM people AS p
WHERE p.company_id IN
(SELECT DISTINCT(c.id)
FROM company AS c
JOIN funding_round AS fr ON c.id=fr.company_id
WHERE c.status = 'closed' AND fr.is_last_round = 1 AND fr.is_first_round = 1)),

total AS
(SELECT person_id, COUNT(instituition)
FROM education AS e
JOIN p_id ON e.person_id=p_id.id
GROUP BY person_id)

SELECT AVG(count)
FROM total;
'''

### Задание 18

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.

In [ ]:
'''
WITH
table_1 AS
(SELECT p.id
FROM people AS p 
JOIN company AS c ON p.company_id=c.id
WHERE c.name = 'Socialnet'),

table_2 AS
(SELECT person_id, COUNT(instituition)
FROM education
GROUP BY person_id),

total AS
(SELECT person_id, COUNT(instituition)
FROM education AS e
JOIN table_1 ON e.person_id=table_1.id
GROUP BY person_id)

SELECT AVG(count)
FROM total;
'''

### Задание 19

Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
'''
SELECT 
f.name AS name_of_fund,
c.name AS name_of_company,
fr.raised_amount AS amount
FROM funding_round AS fr
INNER JOIN investment AS i ON fr.id = i.funding_round_id
INNER JOIN fund AS f ON i.fund_id = f.id
INNER JOIN company AS c ON fr.company_id = c.id
WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2012 AND 2013
AND c.milestones > 6;
'''

### Задание 20

Выгрузите таблицу, в которой будут такие поля:
- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
'''
SELECT c_acquiring.name, 
       a.price_amount, 
       c_acquired.name,
       c_acquired.funding_total, 
       ROUND(a.price_amount/c_acquired.funding_total)
FROM acquisition AS a
LEFT JOIN company AS c_acquiring ON c_acquiring.id = a.acquiring_company_id
LEFT JOIN company AS c_acquired ON c_acquired.id =a.acquired_company_id
WHERE a.price_amount > 0 AND c_acquired.funding_total > 0
ORDER BY a.price_amount DESC, c_acquired.name
LIMIT 10;
'''

### Задание 21

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
'''
SELECT c.name, EXTRACT(MONTH FROM CAST(fr.funded_at AS date))
FROM company AS c
LEFT JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code LIKE 'social'
AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013
AND fr.raised_amount > 0;
'''

### Задание 22

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

In [ ]:
'''
WITH
table_1 AS
(SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month,
COUNT(DISTINCT(f.name)) AS funds
FROM funding_round AS fr
INNER JOIN investment AS i ON fr.id=i.funding_round_id
INNER JOIN fund AS f ON i.fund_id=f.id
WHERE f.country_code = 'USA'
AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN 2010 AND 2013
GROUP BY month),

table_2 AS
(SELECT EXTRACT(MONTH FROM CAST(a.acquired_at AS date)) AS month,
COUNT(a.acquired_company_id) AS count,
SUM(a.price_amount) AS sum
FROM acquisition AS a
WHERE EXTRACT(YEAR FROM CAST(a.acquired_at AS date)) BETWEEN 2010 AND 2013
GROUP BY month)

SELECT table_1.month, table_1.funds, table_2.count, table_2.sum
FROM table_2
JOIN table_1 ON table_2.month = table_1.month;
'''

### Задание 23

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
'''
WITH
inv_2011 AS (SELECT country_code, AVG(funding_total) AS avg_2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
GROUP BY country_code),

inv_2012 AS (SELECT country_code, AVG(funding_total) AS avg_2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
GROUP BY country_code),

inv_2013 AS (SELECT country_code, AVG(funding_total) AS avg_2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
GROUP BY country_code)

SELECT inv_2011.country_code, 
       inv_2011.avg_2011, 
       inv_2012.avg_2012, 
       inv_2013.avg_2013       
FROM inv_2011 
INNER JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013 ON inv_2011.country_code = inv_2013.country_code
ORDER BY inv_2011.avg_2011 DESC;
'''